<a href="https://colab.research.google.com/github/xhiroga/til/blob/main/software-engineering/haystack/_src/sandbox/Haystack%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E8%B3%AA%E7%96%91%E5%BF%9C%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash

nvidia-smi

Tue Sep 26 20:59:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2


In [5]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch


In [6]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,preprocessing,elasticsearch,inference]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Obtaining dependency information for farm-haystack[colab,elasticsearch,inference,preprocessing] from https://files.pythonhosted.org/packages/31/db/e81141e15cecf1abc1238d9aee55f66310274878a034dd96603703620e9c/farm_haystack-1.20.1-py3-none-any.whl.metadata
  Obtaining dependency information for canals==0.7.0 from https://files.pythonhosted.org/packages/08/2f/62e9e455c4cf183ebf6672c47befabb6a36a37a8cdc86256a5632d6ace10/canals-0.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for events from https://files.pythonhosted.org/packages/25/ed/e47dec0626edd468c84c04d97769e7ab4ea6457b7f54dcb3f72b17fcd876/Events-0.5-py3-none-any.whl.metadata
  Obtaining dependency information for posthog from https://files.pythonhosted.org/packages/a7/73/35758818228c70348be4c3c6

In [7]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

# あとで密なRetrieverとともに使うので、文書埋め込みを返すようにする
document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document", return_embedding=True)


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
file_path = "/content/drive/My Drive/Colab Notebooks/assets/justincase-jp_smartphone_insurance_policy_doc.md"

with open(file_path, 'r') as file:
    content = file.read()

print(content[0:100])

<a id="top"></a>
# <u>株式会社justInCase - 通信端末修理費用保険（スマホ保険） 普通保険約款・特約</u>

2018/7/1

- [株式会社justInCase 


In [14]:
documents=[{
  'content': content,
  'meta': {'name': "justincase-jp_smartphone_insurance_policy_doc.md"}
}]
document_store.write_documents(documents=documents, index="document")

In [15]:
from haystack.nodes.retriever import BM25Retriever

bm25_retriever = BM25Retriever(document_store=document_store)

In [16]:
from haystack.nodes import FARMReader

model_ckpt = "deepset/minilm-uncased-squad2" #alternative larger models: deepset/roberta-base-squad2-distilled or deepset/xlm-roberta-large-squad2 or the tiny distilled model: deepset/tinyroberta-squad2
max_seq_length, doc_stride = 384, 128
reader = FARMReader(model_name_or_path=model_ckpt, progress_bar=False,
                    max_seq_len=max_seq_length, doc_stride=doc_stride,
                    return_no_answer=True)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader=reader, retriever=bm25_retriever)

In [18]:
query = "更新案内はいつ送信されますか？"
n_answers = 3
preds = pipe.run(query=query, params={"Retriever": {"top_k": 3, "filters":{}}, "Reader": {"top_k": n_answers}})

print(f"Question: {preds['query']} \n")

for idx in range(n_answers):
    print(f"Answer {idx+1}: {preds['answers'][idx].answer}")
    print(f"Review snippet: ...{preds['answers'][idx].context}...")
    print("\n\n")

Question: 更新案内はいつ送信されますか？ 

Answer 1: 
Review snippet: ...None...



Answer 2: 保険金
Review snippet: ...記1.の事実の発生したときまたは契約者がその発生を知ったときから、当社が前記1.の変更にかかる当社への送信を受信するまでの間に生じた損害に対しては、保険金をお支払いしない場合があります。

<a id="part1_article27"><a>
#### _27条 （保険契約内容の変更）_

この保...



Answer 3: 保険金を
Review snippet: ...1.の事実の発生したときまたは契約者がその発生を知ったときから、当社が前記1.の変更にかかる当社への送信を受信するまでの間に生じた損害に対しては、保険金をお支払いしない場合があります。

<a id="part1_article27"><a>
#### _27条 （保険契約内容の変更）_

この保険...



